In [ ]:
%run ./modules/load_data_module.ipynb

In [ ]:
import pandas as pd
import spacy
import string
import regex as re
import nltk
import ast
import copy
import glob
import advertools as adv
import plotly.graph_objects as go


pd.options.display.max_colwidth = 285



from textblob import TextBlob
from spacymoji import Emoji
from nltk.corpus import stopwords
from urllib.parse import urlparse
from textblob import Word
from sklearn.metrics.pairwise import cosine_similarity


nlp = spacy.load("en_core_web_sm")
emoji = Emoji(nlp)
nlp.add_pipe("emoji", first=True)


stemmer = nltk.SnowballStemmer("english")


nltk.download('wordnet')
nltk.download('omw-1.4')


In [ ]:
tweets_path =glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*tweets.csv')
users_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*users.csv')
replies_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*replies.csv')
quotes_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*quotes.csv')

In [ ]:
tweets = get_df(tweets_path)
replies = get_df(replies_path)
quotes = get_df(quotes_path)
users = get_df(users_path)

In [ ]:
tweet = copy.deepcopy(tweets)
reply = copy.deepcopy(replies)
quote = copy.deepcopy(quotes).fillna('None')

# initial dataframe cleaning

In [ ]:
user = clean_users(users)

In [ ]:
print(len(user), len(users))

# GEN-DATA 

In [ ]:
%run ./modules/load_data_module.ipynb

In [ ]:
tweet['hashtags'] = add_hashtags(tweets)

In [ ]:
tweet['hashtag_count'] = add_nbr_hashtags(tweets)


In [ ]:
tweet['uppercase_count'] = add_uppercase_count(tweet)

In [ ]:
tweet[]

In [ ]:
tweets_update['clean_text'] = add_clean_text(tweets)

In [ ]:
tweets_update['uppercase_count'] = add_uppercase_count(tweets)

In [ ]:
tweets_update['uppercase'] = add_uppercase(tweets)

In [ ]:
tweets_update['emoji'] = add_emoji_list(tweets)

In [ ]:
tweets_update['emoji_group'] = add_emoji_group(tweets)

In [ ]:
tweets_update['clean_text']= add_clean_text(tweets)

In [ ]:
tweets_update['polarity'] = add_polarity(tweets_update)

In [ ]:
tweets_update['all_reply_count'] = add_count_all_reply(tweets, replies)

In [ ]:
tweets_update['total_raction'] = add_count_reaction(tweets_update)

In [ ]:
tweets_update['followers_count'] = users['followers_count'] 

In [ ]:
tweets_update['engagement_score'] = add_engagement_score(tweets_update)

In [ ]:
tweets_update.head(1)

In [ ]:
users.head(1)

In [ ]:
reply_update['hastags'] = add_hastags(replies)

In [ ]:
reply_update['clean_text'] = add_clean_text(replies)

In [ ]:
reply_update['uppercase_count'] = add_uppercase_count(replies)

In [ ]:
reply_update['uppercase'] = add_uppercase(replies)

In [ ]:
reply_update['polarity'] = add_polarity(reply_update)

In [ ]:
reply_update['total_reaction'] = reply_update['retweet_count'] + reply_update['reply_count'] + reply_update['like_count'] + reply_update['quote_count']

In [ ]:
reply_update['emoji'] = add_emoji_list(replies)

In [ ]:
reply_update['emoji_group'] = add_emoji_group(replies)

In [ ]:
reply_update.head(1)

# Replies Basic Stats.

In [ ]:
print('Total number of replies:', len(replies))

In [ ]:
print('Average number of uppercase words in replies:', reply_update['uppercase_count'].mean())

# Tweet's Basic Statistics

In [ ]:
print('Total number of tweets:', len(tweets))

In [ ]:
print('On average number of retweets:', len(replies)/len(tweets) * 100)

In [ ]:
print('Average number of uppercase words in tweets:', tweets_update['uppercase_count'].mean())

In [ ]:
print('Average engagement score of tweets :', tweets_update['engagement_score'].mean())

In [ ]:
tweets_update['engagement_score'].max()

In [ ]:
xs = tweets_update['engagement_score'].dropna()



fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=xs))

fig.update_layout(
    title="Boxplot distribution of engagement score",
    width=700,
    height=400,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()



In [ ]:
def get_count_uppercase(pd_series):
    all_hastags = {}
    for hash_list in pd_series:
        if hash_list:
            for h in hash_list:
                if h in all_hastags.keys():
                    all_hastags[h] += 1
                else:
                    all_hastags[h] = 1
    return all_hastags
    
    
uppercase_count = get_count_uppercase(tweets_update['uppercase'])
total_uppercase = sum(uppercase_count.values())

uppercase_pct = [(k,(v/total_uppercase)*100) for k,v in sorted(uppercase_count.items(), key=lambda item: item[1], reverse=True)]
uppercase_pct[:10]

xs = [x[0] for x in uppercase_pct[:10]]
ys = [x[1] for x in uppercase_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used uppercase words found in Tweets",
    xaxis_title="Top 10 uppercase words found in Tweets",
    yaxis_title="Uppercase words precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

uppercase_count_reply = get_count_uppercase(reply_update['uppercase'])
total_uppercase_reply = sum(uppercase_count_reply.values())

uppercase_reply_pct = [(k,(v/total_uppercase_reply)*100) for k,v in sorted(uppercase_count_reply.items(), key=lambda item: item[1], reverse=True)]
uppercase_reply_pct[:10]

xs = [x[0] for x in uppercase_reply_pct[:10]]
ys = [x[1] for x in uppercase_reply_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used uppercase words found in Replies",
    xaxis_title="Top 10 uppercase words found in Replies",
    yaxis_title="Uppercase words precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
def get_count_hastags(pd_series):
    all_hastags = {}
    for hash_list in pd_series:
        if hash_list:
            for h in hash_list:
                if h in all_hastags.keys():
                    all_hastags[h] += 1
                else:
                    all_hastags[h] = 1
    return all_hastags
    
    
hashtag_count = get_count_hastags(tweets_update['hastags'])
total_hastags = sum(hashtag_count.values())

hashtag_pct = [(k,(v/total_hastags)*100) for k,v in sorted(hashtag_count.items(), key=lambda item: item[1], reverse=True)]
hashtag_pct[:10]

xs = [x[0] for x in hashtag_pct[:10]]
ys = [x[1] for x in hashtag_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used Hashtags in Tweets",
    xaxis_title="Top 10 hashtags",
    yaxis_title="Hashtag precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()


hashtag_count_reply = get_count_hastags(reply_update['hastags'])
total_hastags_reply = sum(hashtag_count_reply.values())

hashtag_reply_pct = [(k,(v/total_hastags_reply)*100) for k,v in sorted(hashtag_count_reply.items(), key=lambda item: item[1], reverse=True)]
hashtag_reply_pct[:10]

xs = [x[0] for x in hashtag_reply_pct[:10]]
ys = [x[1] for x in hashtag_reply_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used Hashtags in Replies",
    xaxis_title="Top 10 hashtags",
    yaxis_title="Hashtag precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()



# Tweet's Emoji Statistics

In [ ]:
def get_overall_emoji_counts(series):
    emojies = series.tolist()
    # print(emojies)
    all_emojies = {}
    
    for em in emojies:
        if len(em)>0:
            for e in em:
            
                if e in all_emojies.keys():
                    all_emojies[e] += 1
                else:
                    all_emojies[e] = 1

    return all_emojies


emoji_count = get_overall_emoji_counts(tweets_update['emoji'])

total = sum(emoji_count.values())

emojie_pct = [(k,(v/total)*100) for k,v in sorted(emoji_count.items(), key=lambda item: item[1], reverse=True)]
emojie_pct[:10]

xs = [x[0] for x in emojie_pct[:10]]
ys = [x[1] for x in emojie_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used emojies in Tweets",
    xaxis_title="Top 10 emojies",
    yaxis_title="Tweets emoji precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()



emoji_count_reply = get_overall_emoji_counts(reply_update['emoji'])

total_reply = sum(emoji_count_reply.values())

emojie_reply_pct = [(k,(v/total_reply)*100) for k,v in sorted(emoji_count_reply.items(), key=lambda item: item[1], reverse=True)]
emojie_reply_pct[:10]

xs = [x[0] for x in emojie_reply_pct[:10]]
ys = [x[1] for x in emojie_reply_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used emojies in Replies",
    xaxis_title="Top 10 emojies",
    yaxis_title="Replies emoji precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
def get_overall_emoji_group(series):
    emojies = series.tolist()
    all_emoji_group = {}
    for em in emojies:
        if len(em)>0:
            for e in em:
                if e[0] in all_emoji_group.keys():
                    all_emoji_group[e[0]] += e[1]
                else:
                    all_emoji_group[e[0]] = e[1]
    return all_emoji_group


emoji_group = get_overall_emoji_group(tweets_update['emoji_group'])
        
total_group = sum(emoji_group.values())

emojie_group_pct = [(k,(v/total_group)*100) for k,v in sorted(emoji_group.items(), key=lambda item: item[1], reverse=True)]
emojie_group_pct[:10]

xs = [x[0] for x in emojie_group_pct[:10]]
ys = [x[1] for x in emojie_group_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used emoji groups in Tweets",
    xaxis_title="Top 10 emoji groups",
    yaxis_title="Tweets emoji group precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()



emoji_group_reply = get_overall_emoji_group(reply_update['emoji_group'])
        
total_group_reply = sum(emoji_group_reply.values())

emojie_group_reply_pct = [(k,(v/total_group_reply)*100) for k,v in sorted(emoji_group_reply.items(), key=lambda item: item[1], reverse=True)]
emojie_group_reply_pct[:10]

xs = [x[0] for x in emojie_group_reply_pct[:10]]
ys = [x[1] for x in emojie_group_reply_pct[:10]]

fig = go.Figure(data=go.Scatter(
    x=xs,
    y=ys,
    mode='markers',
    # marker=dict(size=[40, 60, 80, 100],
    #             color=[0, 1, 2, 3])
))


fig.update_layout(
    title="Most used emoji groups in Replies",
    xaxis_title="Top 10 emoji groups",
    yaxis_title="Replies emoji group precentage",
    width=700,
    height=700,
    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
reply_all_positive_pct = len(tweets_update.loc[tweets_update['polarity']=='positive']) / len(tweets_update) * 100
reply_all_negative_pct = len(tweets_update.loc[tweets_update['polarity']=='negative']) / len(tweets_update) * 100
reply_all_neutral_pct = len(tweets_update.loc[tweets_update['polarity']=='neutral']) / len(tweets_update) * 100

y = [reply_all_positive_pct, reply_all_negative_pct, reply_all_neutral_pct]
x = ['positive', 'negative', 'neutral']

In [ ]:
import plotly.graph_objects as go



# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=x, y=y,
            hovertext=['27% market share', '24% market share', '19% market share'])])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Percentage Tweets Sentiment', width=700, height=700)
fig.show()

# GS DATA

In [ ]:
df_sample = tweets.sample()

In [ ]:
df_sample

In [ ]:
cid = df_sample['conversation_id']
cid

In [ ]:
get_all_reply(int(cid), replies)

In [ ]:
df_sample['text']


In [ ]:
for idx, row in get_all_reply(int(cid), replies).iterrows():
    print(row.text)